# Grabbing Jewel's SnowPit Data

1. Grab pit and format correctly to align with SnowEx database
2. Loop through folder of .csv's and create correct temperature profiles
3. Merge all profiles into one "dictionary"??? geopandas magic dataframe?

GoogleDrive PitDataTemperatures Folder: https://drive.google.com/drive/folders/1SFyBKULqiRLi52yiKO5mxRn5Icsp83Lm?usp=sharing

Test Blob (single temp profile .csv): 1DufHRYtWqxQUFrfqdFx1FViFbaTCkmAV

In [19]:
# set up and select a pit to work with
import pandas as pd
df = pd.read_csv('https://drive.google.com/uc?export=download&id=1DufHRYtWqxQUFrfqdFx1FViFbaTCkmAV', header=None)
df

,0,1,2
0,Location:,County Line,NaN
1,Type:,Open,NaN
2,Easting:,756905,NaN
3,Northing:,4324353,NaN
4,Surveyor:,J. Lund,NaN
5,Time:,3/12/20 9:58,NaN
6,Time Type:,AM,NaN
7,Air Temp:,-2.8,NaN
8,Hs:,111,NaN
9,Ground:,"Rough ground, no info on vegetation.",NaN


Pandas subsetting during imports: we'll read in the snow temp data and meta data separately then transpose, duplicate and combine the data

In [20]:
# grab the snow temperature data
dfdata = pd.read_csv('https://drive.google.com/uc?export=download&id=1DufHRYtWqxQUFrfqdFx1FViFbaTCkmAV', header = 13)
dfdata

,Hs,Temperature,Notes
0,110,-4.4,NaN
1,100,-3.5,NaN
2,90,-3.2,NaN
3,80,-3.1,NaN
4,70,-3.1,NaN
5,60,-3.0,NaN
6,50,-2.8,NaN
7,40,-2.7,NaN
8,30,-2.6,NaN
9,20,-2.2,NaN


In [21]:
# grab the metadata only
dfmeta = df.head(12)
dfmeta

,0,1,2
0,Location:,County Line,NaN
1,Type:,Open,NaN
2,Easting:,756905,NaN
3,Northing:,4324353,NaN
4,Surveyor:,J. Lund,NaN
5,Time:,3/12/20 9:58,NaN
6,Time Type:,AM,NaN
7,Air Temp:,-2.8,NaN
8,Hs:,111,NaN
9,Ground:,"Rough ground, no info on vegetation.",NaN


In [22]:
# transpose the metadata to columns
dftranspose = dfmeta.transpose().head(2)
dftranspose

,0,1,2,3,4,5,6,7,8,9,10,11
0,Location:,Type:,Easting:,Northing:,Surveyor:,Time:,Time Type:,Air Temp:,Hs:,Ground:,Notes:,Wx:
1,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,111,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th..."


In [23]:
# grab column names from the first row
newheader = dftranspose.iloc[0]
newheader

0      Location:
1          Type:
2       Easting:
3      Northing:
4      Surveyor:
5          Time:
6     Time Type:
7      Air Temp:
8            Hs:
9        Ground:
10        Notes:
11           Wx:
Name: 0, dtype: object

In [24]:
# remove the column names row
dftranspose = dftranspose[1:]
dftranspose

,0,1,2,3,4,5,6,7,8,9,10,11
1,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,111,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th..."


In [25]:
# add column names to transposed metadata
dftranspose.columns = newheader
dftranspose

,Location:,Type:,Easting:,Northing:,Surveyor:,Time:,Time Type:,Air Temp:,Hs:,Ground:,Notes:,Wx:
1,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,111,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th..."


In [26]:
# duplicate rows of metadata to length of snow pit temperature profile
dfdup = dftranspose.reindex(dftranspose.index.repeat(len(dfdata))).reset_index(drop=True)
dfdup

,Location:,Type:,Easting:,Northing:,Surveyor:,Time:,Time Type:,Air Temp:,Hs:,Ground:,Notes:,Wx:
0,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,111,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th..."
1,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,111,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th..."
2,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,111,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th..."
3,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,111,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th..."
4,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,111,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th..."
5,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,111,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th..."
6,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,111,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th..."
7,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,111,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th..."
8,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,111,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th..."
9,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,111,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th..."


In [27]:
# merge dfdata with dfdup to make a complete df of snow profile temps
dfmerged = pd.concat([dfdup, dfdata], axis = 1)
del dfmerged['Hs:'] # remove the total HS measurement column
dfmerged

,Location:,Type:,Easting:,Northing:,Surveyor:,Time:,Time Type:,Air Temp:,Ground:,Notes:,Wx:,Hs,Temperature,Notes
0,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th...",110,-4.4,NaN
1,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th...",100,-3.5,NaN
2,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th...",90,-3.2,NaN
3,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th...",80,-3.1,NaN
4,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th...",70,-3.1,NaN
5,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th...",60,-3.0,NaN
6,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th...",50,-2.8,NaN
7,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th...",40,-2.7,NaN
8,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th...",30,-2.6,NaN
9,County Line,Open,756905,4324353,J. Lund,3/12/20 9:58,AM,-2.8,"Rough ground, no info on vegetation.",Temperature 9:58-10:10.,"Calm, mostly cloudy, but sun still shining (th...",20,-2.2,NaN


Set new column names and reorder to align with SnowEx database column headers

- Should we do two dataframes to match the database (PointData and LayerData)? I ask because Jewel recorded Type (tree cover), Ground, Notes, and WX data that seems like it might fit better in the Point Data information?

In [55]:
# grab the database column names for LayerData (in correct order)
db_colnames = ["site_name", "date", "time_created", "time_updated", "id", "doi", "date_accessed", "instrument", "type", "units", "...", "geom", "time", "depth", "site_id", 
               "bottom_depth", "comments", "sample_a", "sample_b", "sample_c", "value"]
db_colnames

['site_name',
 'date',
 'time_created',
 'time_updated',
 'id',
 'doi',
 'date_accessed',
 'instrument',
 'type',
 'units',
 '...',
 'geom',
 'time',
 'depth',
 'site_id',
 'bottom_depth',
 'comments',
 'sample_a',
 'sample_b',
 'sample_c',
 'value']

In [56]:
# reorder dfmerged to match database order and add blank columns to fill
dfreorder = pd.DataFrame(columns = db_colnames)
dfreorder

,site_name,date,time_created,time_updated,id,doi,date_accessed,instrument,type,units,...,geom,time,depth,site_id,bottom_depth,comments,sample_a,sample_b,sample_c,value


In [57]:
# add Jewel's pit data into appropriate columns
dfreorder['depth'] = dfmerged['Hs']
dfreorder['value'] = dfmerged['Temperature']
dfreorder['site_id'] = dfmerged['Location:']
dfreorder['type'] = 'temperature'
dfreorder['site_name'] = 'Grand Mesa'
dfreorder

,site_name,date,time_created,time_updated,id,doi,date_accessed,instrument,type,units,...,geom,time,depth,site_id,bottom_depth,comments,sample_a,sample_b,sample_c,value
0,Grand Mesa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,NaN,110,County Line,NaN,NaN,NaN,NaN,NaN,-4.4
1,Grand Mesa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,NaN,100,County Line,NaN,NaN,NaN,NaN,NaN,-3.5
2,Grand Mesa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,NaN,90,County Line,NaN,NaN,NaN,NaN,NaN,-3.2
3,Grand Mesa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,NaN,80,County Line,NaN,NaN,NaN,NaN,NaN,-3.1
4,Grand Mesa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,NaN,70,County Line,NaN,NaN,NaN,NaN,NaN,-3.1
5,Grand Mesa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,NaN,60,County Line,NaN,NaN,NaN,NaN,NaN,-3.0
6,Grand Mesa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,NaN,50,County Line,NaN,NaN,NaN,NaN,NaN,-2.8
7,Grand Mesa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,NaN,40,County Line,NaN,NaN,NaN,NaN,NaN,-2.7
8,Grand Mesa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,NaN,30,County Line,NaN,NaN,NaN,NaN,NaN,-2.6
9,Grand Mesa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,NaN,20,County Line,NaN,NaN,NaN,NaN,NaN,-2.2


In [72]:
# split date/time to separate columns
date_time = dfmerged['Time:'].str.split(expand=True)
dfreorder['date'] = date_time[0]
dfreorder['time'] = date_time[1]
dfreorder

,site_name,date,time_created,time_updated,id,doi,date_accessed,instrument,type,units,...,geom,time,depth,site_id,bottom_depth,comments,sample_a,sample_b,sample_c,value
0,Grand Mesa,3/12/20,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,9:58,110,County Line,NaN,NaN,NaN,NaN,NaN,-4.4
1,Grand Mesa,3/12/20,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,9:58,100,County Line,NaN,NaN,NaN,NaN,NaN,-3.5
2,Grand Mesa,3/12/20,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,9:58,90,County Line,NaN,NaN,NaN,NaN,NaN,-3.2
3,Grand Mesa,3/12/20,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,9:58,80,County Line,NaN,NaN,NaN,NaN,NaN,-3.1
4,Grand Mesa,3/12/20,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,9:58,70,County Line,NaN,NaN,NaN,NaN,NaN,-3.1
5,Grand Mesa,3/12/20,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,9:58,60,County Line,NaN,NaN,NaN,NaN,NaN,-3.0
6,Grand Mesa,3/12/20,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,9:58,50,County Line,NaN,NaN,NaN,NaN,NaN,-2.8
7,Grand Mesa,3/12/20,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,9:58,40,County Line,NaN,NaN,NaN,NaN,NaN,-2.7
8,Grand Mesa,3/12/20,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,9:58,30,County Line,NaN,NaN,NaN,NaN,NaN,-2.6
9,Grand Mesa,3/12/20,NaN,NaN,NaN,NaN,NaN,NaN,temperature,NaN,...,NaN,9:58,20,County Line,NaN,NaN,NaN,NaN,NaN,-2.2


# To Do:
- Add Aji's geometry script in
- Create PointData dataframe with ground roughness, tree cover, etc data
- Create loop to run through all .csv files and append to mega dataframe